In [96]:
from datetime import datetime

class Bcolors:
    """A class with names for ANSI escape sequences defining colors and font styles on terminal.
       Source: https://stackoverflow.com/questions/287871/how-do-i-print-colored-text-to-the-terminal
    """
    HEADER =    '\033[95m'
    OKBLUE =    '\033[94m'
    OKCYAN =    '\033[96m'
    OKGREEN =   '\033[92m'
    WARNING =   '\033[93m'
    FAIL =      '\033[91m'
    ENDC =      '\033[0m'
    BOLD =      '\033[1m'
    UNDERLINE = '\033[4m'


class Astro:
    """A class used to manage astro signs.
    Class attribute 'symbols' is a dictionary that defines names for Unicodes representing arial signs.
    """

    symbols = {
        'Aries':       '\U00002648',
        'Taurus':      '\U00002649',
        'Gemini':      '\U0000264A',
        'Cancer':      '\U0000264B',
        'Leo':         '\U0000264C',
        'Virgo':       '\U0000264D',
        'Libra':       '\U0000264E',
        'Scorpius':    '\U0000264F',
        'Sagittarius': '\U00002650',
        'Capricorn':   '\U00002651',
        'Aquarius':    '\U00002652',
        'Pisces':      '\U00002653'
    }

    def sign_name(month, day):
        """Returns a string with astro sign name for given month and day."""
        if month == 1:    astro_sign = 'Capricorn'   if (day < 20) else 'Aquarius'
        elif month == 2:  astro_sign = 'Aquarius'    if (day < 19) else 'Pisces'
        elif month == 3:  astro_sign = 'Pisces'      if (day < 21) else 'Aries'
        elif month == 4:  astro_sign = 'Aries'       if (day < 20) else 'Taurus'
        elif month == 5:  astro_sign = 'Taurus'      if (day < 21) else 'Gemini'
        elif month == 6:  astro_sign = 'Gemini'      if (day < 21) else 'Cancer'
        elif month == 7:  astro_sign = 'Cancer'      if (day < 23) else 'Leo'
        elif month == 8:  astro_sign = 'Leo'         if (day < 23) else 'Virgo'
        elif month == 9:  astro_sign = 'Virgo'       if (day < 23) else 'Libra'
        elif month == 10: astro_sign = 'Libra'       if (day < 23) else 'Scorpius'
        elif month == 11: astro_sign = 'Scorpius'    if (day < 22) else 'Sagittarius'
        elif month == 12: astro_sign = 'Sagittarius' if (day < 22) else 'Capricorn'
        return astro_sign


class Pesel:
    """A class used to manage PESEL number.
    The PESEL validation algorithm is implemented on the basis of information from the website
    https://obywatel.gov.pl/pl/dokumenty-i-dane-osobowe/czym-jest-numer-pesel.
    """

    def __init__(self, pesel):
        """Initializes 'pesel' attribute of the created object with the value of the argument.
        If the argument is not correct, prints a warning
        and initializes 'pesel' attribute with an empty string.
        """
        self.set_pesel(pesel)

    def __str__(self):
        """Returns a string representing Pesel object. The string consists of
        PESEL number and data that is derived from it (birthday, gender, astro sign).
        In case of empty 'pesel' attribute, returns the string 'No PESEL value'.
        """
        if self.pesel == "":
            return Bcolors.BOLD + "No PESEL value." + Bcolors.ENDC
        birthday = f"{self.get_birthday().strftime('%Y-%m-%d')}"
        astro_symbol = Astro.symbols[Astro.sign_name(self.get_month(), self.get_day())]
        return Bcolors.BOLD + f"{self.pesel}, " + \
               Bcolors.ENDC + Bcolors.OKBLUE + \
               f"{birthday}, {self.get_gender()}  " + Bcolors.ENDC + \
               astro_symbol

    def set_pesel(self, pesel):
        """Sets the 'pesel' attribute of the created object with the value of the argument.
        If the argument is not correct, prints a warning
        and sets 'pesel' attribute with an empty string.
        """
        warn_text = Bcolors.FAIL + \
                    f"WARNING: class Pesel, method set_pesel(\"{pesel}\") - "
        if len(pesel) != 11 or not pesel.isdecimal():
            self.pesel = ""
            print(warn_text + "wrong characters or length." + Bcolors.ENDC)
            return
        self.pesel = pesel
        if self.calculate_control_sum() != int(pesel[10]):
            self.pesel = ""
            print(warn_text + "wrong control sum." + Bcolors.ENDC)
            return
        try:
            datetime(self.get_year(), self.get_month(), self.get_day())
        except:
            self.pesel = ""
            print(warn_text + "incorrect date." + Bcolors.ENDC)

    def get_year(self):
        """Gets the year of birth as int.
        If the 'pesel' attribute contains an empty string, the method returns None.
        """
        if self.pesel == "":
            return None
        begining = ["19", "20", "21", "22", "18"]
        return int(begining[int(self.pesel[2]) // 2] + self.pesel[0:2])

    def get_month(self):
        """Gets the month of birth as int.
        If the 'pesel' attribute contains an empty string, the method returns None.
        """
        if self.pesel == "":
            return None
        return int(self.pesel[2:4]) - (int(self.pesel[2]) // 2) * 20

    def get_day(self):
        """Gets the day of birth as int.
        If the 'pesel' attribute contains an empty string, the method returns None.
        """
        if self.pesel == "":
            return None
        return int(self.pesel[4:6])

    def get_birthday(self):
        """Gets the birthday as datatime.
        If the year, month, day do not form correct date, returns None.
        """
        try:
            return datetime(self.get_year(), self.get_month(), self.get_day())
        except:
            return None

    def get_gender(self):
        """Gets the gender: "F" for female, "M" for male.
        If the 'pesel' attribute contains an empty string, the method returns None.
        """
        if self.pesel == "":
            return None
        return "F" if self.pesel[9] in {'0', '2', '4', '6', '8'} else "M"

    def calculate_control_sum(self):
        """Calculates the control sum.
        If the 'pesel' attribute contains an empty string, the method returns None.
        """
        if self.pesel == "":
            return None
        ct = [1, 3, 7, 9, 1, 3, 7, 9, 1, 3]
        sum = 0
        for i in range(10):
            sum += (int(self.pesel[i]) * ct[i])
        return 10 - int(str(sum)[-1])

print("Classes: Bcolors, Astro, Pesel created.")

Classes: Bcolors, Astro, Pesel created.


<br>

## Task 1 - abstraction

In this task, you are to learn how to use the `Pesel` class without understanding its code. The only thing you will need to do is read the description of the class and its methods.

First, please execute the command
```python
help(Pesel)
```
and read the description there. Where did the descriptions of the methods and class come from? Please look in the code above.

In [47]:
help(Pesel)

Help on class Pesel in module __main__:

class Pesel(builtins.object)
 |  Pesel(pesel)
 |  
 |  A class used to manage PESEL number.
 |  The PESEL validation algorithm is implemented on the basis of information from the website
 |  https://obywatel.gov.pl/pl/dokumenty-i-dane-osobowe/czym-jest-numer-pesel.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, pesel)
 |      Initializes 'pesel' attribute of the created object with the value of the argument.
 |      If the argument is not correct, prints a warning
 |      and initializes 'pesel' attribute with an empty string.
 |  
 |  __str__(self)
 |      Returns a string representing Pesel object. The string consists of
 |      PESEL number and data that is derived from it (birthday, gender, astro sign).
 |      In case of empty 'pesel' attribute, returns the string 'No PESEL value'.
 |  
 |  calculate_control_sum(self)
 |      Calculates the control sum.
 |      If the 'pesel' attribute contains an empty string, the method returns No



"Cheat Sheet" - simplified UML diagram of the `Pesel` class:
```
                    +----------------------+                  
                    |        Pesel         |                  
                    |----------------------|                  
                    | pesel                |  
                    |----------------------|                  
                    | __init__             |                  
                    | __str__              |                  
                    | calculate_control_sum|                  
                    | get_birthday         |                  
                    | get_day              |                  
                    | get_gender           |                  
                    | get_month            |                  
                    | get_year             |                  
                    | set_pesel            |                  
                    +----------------------+                  

```

Now, please test the `Pesel` class. I suggest separating each task with the following instruction:
```python
print("\n----------------------------------")
```

1. Create an object of type `Pesel` for the PESEL `"49080134328"` and print this object.
2. Create a list `pesel_objects` of `Pesel` objects based on the list `pesel_strings`:
```python
pesel_strings = ["49080134328", "65120498991", "30892208062", "13311807941", "07413106613",
                    "22681417098", "55080426688", "12345", "06270227781", "06270227782", "06273227788"
                   ]
```
3. Print the elements of the `pesel_objects` list. Pay attention to what happens when there is an invalid PESEL.
4. Print the `pesel` attribute for each object in the `pesel_objects` list.
5. For each object in the `pesel_objects` list, print the birth year encoded in the PESEL, gender, birth date, and the calculated control sum.
6. Change the PESEL of the last object in the `pesel_objects` list, e.g., to `"97102351398"`. Do not create a new object, do not change the `pesel` attribute directly, but use the appropriate method. Print all the objects in the list again.

In [97]:
class Pesel:
  pesel_1 = Pesel("49080134328")
  print(f"PESEL: {pesel_1}")
  print("\n----------------------------------")

  pesel_strings = ["49080134328", "65120498991", "30892208062", "13311807941", "07413106613",
                  "22681417098", "55080426688", "12345", "06270227781", "06270227782", "06273227788"
                  ]
  pesel_objects = [Pesel(pesel_notlist) for pesel_notlist in pesel_strings]
  print("\n----------------------------------")

  for pesel_2 in pesel_objects:
    print(f"PESEL: {pesel_2}")
  print("\n----------------------------------")

  for ell in pesel_objects:
    print(ell.pesel)
  print("\n----------------------------------")

  for atr in pesel_objects:
    print(f"PESEL:{atr.pesel}, sex: {atr.get_gender()}, birthday: {atr.get_birthday()}, sum: {atr.calculate_control_sum()}")
  print("\n----------------------------------")

  pesel_objects[-1].set_pesel("97102351398")
  for chpesel in pesel_objects:
    print(chpesel)


PESEL: 49080134328, 1949-08-01, F  ♌

----------------------------------

----------------------------------
PESEL: 49080134328, 1949-08-01, F  ♌
PESEL: 65120498991, 1965-12-04, M  ♐
PESEL: 30892208062, 1830-09-22, F  ♍
PESEL: 13311807941, 2013-11-18, F  ♏
PESEL: 07413106613, 2107-01-31, M  ♒
PESEL: 22681417098, 2222-08-14, M  ♌
PESEL: 55080426688, 1955-08-04, F  ♌
PESEL: No PESEL value.
PESEL: 06270227781, 2006-07-02, F  ♋
PESEL: No PESEL value.
PESEL: No PESEL value.

----------------------------------
49080134328
65120498991
30892208062
13311807941
07413106613
22681417098
55080426688

06270227781



----------------------------------
PESEL:49080134328, sex: F, birthday: 1949-08-01 00:00:00, sum: 8
PESEL:65120498991, sex: M, birthday: 1965-12-04 00:00:00, sum: 1
PESEL:30892208062, sex: F, birthday: 1830-09-22 00:00:00, sum: 2
PESEL:13311807941, sex: F, birthday: 2013-11-18 00:00:00, sum: 1
PESEL:07413106613, sex: M, birthday: 2107-01-31 00:00:00, sum: 3
PESEL:22681417098, sex: M, bir

<br>

## Task 2 - creating a class and objects, checking the correctness of attributes

Create the `Person` class. It will be used to store data about people. For now, there should be only one attribute of the object in this class: `first_name`. Create it in the `__init__` method. Remember that the `__init__` method will have two parameters: `self` and a parameter through which we will pass the name, e.g. `first_name`.

Test the class by creating objects with the names `"Tom"`, `"Tom1"`, `"tom"`. Print the `first_name` attribute of the created objects.

Note: the `first_name` parameter is different from the `first_name` attribute, despite the same name. In the `__init__` method, we refer to the argument by writing `first_name`, and to the attribute by writing `self.first_name`.

In [98]:
# The first version of class Person
class Person:
  def __init__(self, first_name):
    self.first_name = first_name
obj1 = Person("Tom")
obj2 = Person("Tom1")
obj3 = Person("tom")
toms = [obj1, obj2, obj3]
for atr in toms:
  print(atr.first_name)


Tom
Tom1
tom


<br>

Note that names should only contain letters, with the first character being uppercase. This is not checked when the object is created, but it should be.

<div class="alert alert-block alert-info">
The class should provide a means to prevent or at least make it difficult to create an invalid object.

</div>

Write a function `is_correct_name` that checks whether a given string contains no obvious errors that would prevent it from being a name, i.e. whether it contains only letters and whether the first character is uppercase. The function should return a boolean value. In case the string cannot be a name, the function should print a warning to the screen, e.g.
```
WARNING: functiom is_correct_name("Tom1") - non-letter or first character not uppercase
```
Test the function.

Hint: use the `isalpha` method applied to the string.

In [99]:
# Function 'is_correct_name'
def is_correct_name(name):
  res = name.isalpha()
  if name[0].islower() or res == False:
    print(f'WARNING: function is_correct_name ("{name}") - non-letter or first character not uppercase')
    return False
  else:
    return True
is_correct_name("Tom")
is_correct_name("Tom1")
is_correct_name("tom")

False

<br>

Write a second version of the `Person` class. In the `__init__` method, use the `is_correct_name` function. In the case of an incorrect name, assign an empty string to the `first_name` attribute.

Additionally, in the `__init__` method, create a second attribute `last_name` in the same way. Test the class.

In [100]:
# The second version of class Person
class Person:
  def __init__(self, first_name, last_name):
    self.first_name = first_name if is_correct_name(first_name) else ""
    self.last_name = last_name if is_correct_name(last_name) else ""
obj1 = Person("Tom","Brandt")
obj2 = Person("Tom1","Brandt")
obj3 = Person("tom","brandt")
obj4 = Person("Tom","bran4t")
toms = [obj1, obj2, obj3, obj4]
for atr in toms:
  print(f"First name: {atr.first_name}, last name: {atr.last_name}")



First name: Tom, last name: Brandt
First name: , last name: Brandt
First name: , last name: 
First name: Tom, last name: 


<br>

The `is_correct_name` function currently exists outside the `Person` class. This is not the best solution, because it is closely related to the `Person` class and used in it. However, there is no need for it to be called for the object - after creating the object, we will no longer check the correctness of the name.

In such a case, the `is_correct_name` function should be moved to the `Person` class and created as a method of the class, not the object. To do this, simply paste the function into the class, correcting the indentation. You do not have to change anything in the function itself (maybe only the content of the warning printed to the screen):
```
WARNING: class Person, method is_correct_name("Tom1") - non-letter or first character not uppercase
```
However, the `__init__` method should be corrected. Instead of `is_correct_name(first_name)`, it must now be `Person.is_correct_name(first_name)`.

In [101]:
if "is_correct_name" in globals():
    del is_correct_name

# The third version of class Person
class Person:
  def is_correct_name(name):
    res = name.isalpha()
    if name[0].islower() or res == False:
      print(f'WARNING: function is_correct_name ("{name}") - non-letter or first character not uppercase')
      return False
    else:
      return True
  def __init__(self, first_name, last_name):
    self.first_name = first_name if Person.is_correct_name(first_name) else ""
    self.last_name = last_name if Person.is_correct_name(last_name) else ""
obj1 = Person("Tom","Brandt")
obj2 = Person("Tom1","Brandt")
obj3 = Person("tom","brandt")
obj4 = Person("Tom","bran4t")
toms = [obj1, obj2, obj3, obj4]
for atr in toms:
  print(f"First name: {atr.first_name}, last name: {atr.last_name}")



First name: Tom, last name: Brandt
First name: , last name: Brandt
First name: , last name: 
First name: Tom, last name: 


<br>

## Task 3 - Printing an object

Test the code:
```python
person = Person("Tom", "Joe")
print(person)
```
What was printed?

In [102]:
person = Person("Tom", "Joe")
print(person)



---

Usually, when we want to print an object, we don't want to see a line like:
```
<__main__.Person object at 0x061CE4D8>
```
We would prefer to see something like:
```
first name: Tom
last name: Joe
```

To achieve this, define the `__str__` method in the `Person` class. It must have one argument: `self`. This method should return a string created using the `first_name` and `last_name` attributes. Test the object printing again.

Note: A similar method has been defined for the `Pesel` class. The returned string looks complicated because it includes text formatting (colors, bold font, zodiac signs). You don't need to format the text here.

In [103]:
# The fourth version of class Person
class Person:
  def is_correct_name(self, name):
    res = name.isalpha()
    if name[0].islower() or res == False:
      print(f'WARNING: function is_correct_name ("{name}") - non-letter or first character not uppercase')
      return False
    else:
      return True
  def __init__(self, first_name, last_name):
    self.first_name = first_name if self.is_correct_name(first_name) else ""
    self.last_name = last_name if self.is_correct_name(last_name) else ""
  def __str__(self):
    return f"First name: {self.first_name}, \nLast name: {self.last_name}"
obj1 = Person("Tom","Brandt")
obj2 = Person("Tom1","Brandt")
obj3 = Person("tom","brandt")
obj4 = Person("Tom","bran4t")
toms = [obj1, obj2, obj3, obj4]
for atr in toms:
  print(str(atr))



First name: Tom, 
Last name: Brandt
First name: , 
Last name: Brandt
First name: , 
Last name: 
First name: Tom, 
Last name: 




## Task 4 - Encapsulation - Set and Get Methods

Note that currently, if we create an object with an incorrect first name or last name, to change it we would have to directly modify the corresponding attribute. This is dangerous because it's easy to forget to use the `Person.is_correct_name` method beforehand to detect potential errors.

<div class="alert alert-block alert-info">
The class should provide means to prevent or at least make it difficult to create an invalid object.
</div>

Even if we don't forget, it's still inconvenient because we would have to write code like this:
```python
person = Person("ada", "Joe")
name = "Ada"
if Person.is_correct_name(name):
    person.first_name = name
else:
    person.first_name = ""
```

<div class="alert alert-block alert-info">
It's better if the user of the class doesn't directly modify the object's attributes. The class should provide means to do this through methods.
</div>

Therefore, it's a good idea to write setters, i.e., methods `set_first_name` and `set_last_name`, which perform validation and appropriately modify the attributes. Write them. Be sure to use them in the `__init__` method instead of the existing code.

<div class="alert alert-block alert-info">
Avoid repeating code at all costs.
</div>

Also, write getters, i.e., methods `get_first_name` and `get_last_name`, which return the values of the respective attributes.

Hint: If you're unsure what arguments the setters or getters should have, take a look at the code of the `Pesel` class.

**Benefit**: If the creator of the `Person` class decides to change its implementation (e.g., storing the first and last names not in separate attributes but as a list of two strings), users of this class who never accessed the `first_name` or `last_name` attributes directly, but always through methods, will not have to change their code — everything will still work. This is why getters are often written, even though they might seem unnecessary.

Remember to test the code.

In [104]:
# The fifth version of class Person
class Person:
  def is_correct_name(self, name):
    res = name.isalpha()
    if name[0].islower() or res == False:
      print(f'WARNING: function is_correct_name ("{name}") - non-letter or first character not uppercase')
      return False
    else:
      return True
  def __init__(self, first_name, last_name):
    self.set_first_name(first_name)
    self.set_last_name(last_name)
  def set_first_name(self, first_name):
    if self.is_correct_name(first_name):
      self.first_name = first_name
    else:
      self.first_name = ""
  def set_last_name(self, last_name):
    if self.is_correct_name(last_name):
      self.last_name = last_name
    else:
      self.last_name = ""
  def get_first_name(self):
    return self.first_name
  def get_last_name(self):
    return self.last_name
  def __str__(self):
    return f"First name: {self.first_name}, \nLast name: {self.last_name}"
obj1 = Person("Tom","Brandt")
obj2 = Person("Tom1","Brandt")
obj3 = Person("tom","brandt")
obj4 = Person("Tom","bran4t")
toms = [obj1, obj2, obj3, obj4]
for atr in toms:
  print(str(atr))


First name: Tom, 
Last name: Brandt
First name: , 
Last name: Brandt
First name: , 
Last name: 
First name: Tom, 
Last name: 


<br>

## Task 5 - composition

Composition occurs when one object has attributes that are objects of another class. This is a common phenomenon.

Add the `pesel` attribute to the `Person` class, which is an object of the `Pesel` class. Be sure to write a setter and getter for it. Use the setter to create the attribute in the `__init__` method.

Update the `__str__` method. Remember to test the code.

In [105]:
# The sixth version of class Person
class Pesel:
    def __init__(self, pesel_number):
        if self.is_valid_pesel(pesel_number):
            self.pesel_number = pesel_number
        else:
            self.pesel_number = ""

    def is_valid_pesel(self, pesel_number):
        return len(pesel_number) == 11 and pesel_number.isdigit()

    def __str__(self):
        if self.pesel_number == "":
            return "No valid PESEL"
        return f"PESEL: {self.pesel_number}"

    def get_year(self):
      if self.pesel_number:
        year_prefix = self.pesel_number[:2]
        year = int(year_prefix)
        if year < 20:
          return 2000 + year
        else:
          return 1900 + year
      return None

class Person:
    def is_correct_name(self, name):
        res = name.isalpha()
        if name[0].islower() or res == False:
            print(f'WARNING: function is_correct_name ("{name}") - non-letter or first character not uppercase')
            return False
        else:
            return True

    def __init__(self, first_name, last_name, pesel):
        self.set_first_name(first_name)
        self.set_last_name(last_name)
        self.set_pesel(pesel)

    def set_pesel(self, pesel):
        self.pesel = Pesel(pesel)
        if self.pesel.pesel_number == "":
            self.pesel = None

    def set_first_name(self, first_name):
        if self.is_correct_name(first_name):
            self.first_name = first_name
        else:
            self.first_name = ""

    def set_last_name(self, last_name):
        if self.is_correct_name(last_name):
            self.last_name = last_name
        else:
            self.last_name = ""

    def get_first_name(self):
        return self.first_name

    def get_last_name(self):
        return self.last_name

    def get_pesel(self):
        return self.pesel

    def __str__(self):
        pesel_str = str(self.pesel) if self.pesel else "No valid PESEL"
        return f"First name: {self.first_name}, \nLast name: {self.last_name}, \n{pesel_str}"

    def is_male(self):
      if self.pesel and self.pesel.pesel_number != "":
          return int(self.pesel.pesel_number[9]) % 2 == 1
      return False

    def get_year(self):
      if self.pesel_number:
          year_prefix = self.pesel_number[:2]
          year = int(year_prefix)
          if year < 20:
              return 2000 + year
          else:
              return 1900 + year
      return None


obj1 = Person("Tom", "Brandt", "49080134328")
obj2 = Person("Tom1", "Brandt", "12345")
obj3 = Person("tom", "brandt", "55080426688")
obj4 = Person("Tom", "bran4t", "06270227782")

toms = [obj1, obj2, obj3, obj4]
for atr in toms:
    print(str(atr))



First name: Tom, 
Last name: Brandt, 
PESEL: 49080134328
First name: , 
Last name: Brandt, 
No valid PESEL
First name: , 
Last name: , 
PESEL: 55080426688
First name: Tom, 
Last name: , 
PESEL: 06270227782


<br>

## Task 6 - Using the `Person` class

Using the following lists:

```python
pesels = ("96041369189", "02241916791", "03290941521", "99050283147", "98100144331", "98030692285", "01262733619", "00311262898", "98090516376", "97102351398")
first_names = ("Bolesława", "Witold", "Subisława", "Klaudyna", "Aleks", "Delfina", "Samuel", "Otto", "Zygfryd", "Dobrogost")
last_names =("Kubas", "Miotk", "Piętka", "Boratyńska", "Wojtkiewicz", "Moskała", "Sobiech", "Toporek", "Mrożek", "Skałecki")
```

Create a list of `persons` of `Person` objects, then print it out.

In [106]:
pesels = ("96041369189", "02241916791", "03290941521", "99050283147", "98100144331", "98030692285", "01262733619", "00311262898", "98090516376", "97102351398")
first_names = ("Bolesława", "Witold", "Subisława", "Klaudyna", "Aleks", "Delfina", "Samuel", "Otto", "Zygfryd", "Dobrogost")
last_names =("Kubas", "Miotk", "Piętka", "Boratyńska", "Wojtkiewicz", "Moskała", "Sobiech", "Toporek", "Mrożek", "Skałecki")
persons = []
for i in range(len(pesels)):
  pesel = pesels[i]
  first_name = first_names[i]
  last_name = last_names[i]
  people = Person(first_name, last_name, pesel)
  persons.append(people)
for people in persons:
    print(str(people))
    print()



First name: Bolesława, 
Last name: Kubas, 
PESEL: 96041369189

First name: Witold, 
Last name: Miotk, 
PESEL: 02241916791

First name: Subisława, 
Last name: Piętka, 
PESEL: 03290941521

First name: Klaudyna, 
Last name: Boratyńska, 
PESEL: 99050283147

First name: Aleks, 
Last name: Wojtkiewicz, 
PESEL: 98100144331

First name: Delfina, 
Last name: Moskała, 
PESEL: 98030692285

First name: Samuel, 
Last name: Sobiech, 
PESEL: 01262733619

First name: Otto, 
Last name: Toporek, 
PESEL: 00311262898

First name: Zygfryd, 
Last name: Mrożek, 
PESEL: 98090516376

First name: Dobrogost, 
Last name: Skałecki, 
PESEL: 97102351398



<br>

Print only those objects from the `persons` list that represent males.

In [107]:
for person in persons:
    if person.is_male():
        print(str(person))
        print()




First name: Witold, 
Last name: Miotk, 
PESEL: 02241916791

First name: Aleks, 
Last name: Wojtkiewicz, 
PESEL: 98100144331

First name: Samuel, 
Last name: Sobiech, 
PESEL: 01262733619

First name: Otto, 
Last name: Toporek, 
PESEL: 00311262898

First name: Zygfryd, 
Last name: Mrożek, 
PESEL: 98090516376

First name: Dobrogost, 
Last name: Skałecki, 
PESEL: 97102351398



<br>

Print only those objects from the `persons` list that represent people born after the year 2000.

In [108]:
for young in persons:
    if young.get_pesel().get_year() > 2000:
        print(str(young))
        print()

First name: Witold, 
Last name: Miotk, 
PESEL: 02241916791

First name: Subisława, 
Last name: Piętka, 
PESEL: 03290941521

First name: Samuel, 
Last name: Sobiech, 
PESEL: 01262733619

